In [50]:
import pandas as pd
from src.utils import *
import json

Read the raw file from Stack Exchange Data Explorer

In [51]:
df = pd.read_csv('../data/DataExplorerExport4.csv')

Read the already annotated CSV file for keeping those annotations

In [52]:
filtered_annotated_df = pd.read_csv('../data/DataExp4_filtered_test_val_annotated.csv')

In [53]:
df = df[~df["Title"].isna()]

In [54]:
df['Title'].nunique()

35673

In [55]:
df.head()

,QuerySetId,Title,Description,QueryBody,CreationDate,Views,Votes
0,14342,What's my Pundit badge progress?,How many of my comments have scores of 5 and h...,-- What's my Pundit badge progress?\n-- How ma...,2010-11-17 09:20:44Z,41810,30
7,1116781,TOP 10,NaN,SELECT OwnerUserId\nFROM Posts,2019-09-30 15:03:18Z,23,0
9,139893,Questions by tags,NaN,Select * from posttags,2013-09-30 04:27:43Z,3510,0
12,14517,Keystrokes per rep point (lower limit),Ranks users with over 1000 reputation points b...,-- Keystrokes per rep point\n-- Ranks users wi...,2010-11-20 00:38:02Z,2838,0
13,14542,Nick Craver comparison.,What questions does Nick Craver have a higher/...,-- Nick Craver comparison.\n-- What questions ...,2010-11-20 20:34:20Z,2522,1


In [56]:
# oh, and whenever there are multiple queries for a title - 
# we'd take the latest one that passes through all our filters

# we can write rules to filter out titles that we can tell aren't related to the queries 
# - like numbers that do not appear in the query, columns that aren't being mentioned 
# (for example if title mentions title of posts, title column should be mentioned, etc), 
# keywords (if title has "contain", perhaps query should have LIKE or %)

In [57]:
df['CreationDate']=df.CreationDate.apply(lambda x:x[:-1])

In [58]:
df['CreationDate']=pd.to_datetime(df.CreationDate)

In [59]:
# df=df.sort_values(['CreationDate']).drop_duplicates('Title',keep='last')

Rows to keep because we annotated them

In [60]:
filtered_annotated_df = filtered_annotated_df[~filtered_annotated_df["Type of Fix"].isna()]
print(f"Number of annotated row: {len(filtered_annotated_df)}")

Number of annotated row: 2012


In [61]:
filtered_annotated_df

,Unnamed: 0,QuerySetId,Title,Description,QueryBody,CreationDate,Fixed Title,Fixed QueryBody,Type of Fix,Fixer,Faruk_Error
0,1564873,1337675,Number of Help and Improvement Queries,NaN,"SELECT CreationDate, COUNT(*) FROM ReviewTask...",2020-11-28 16:13:32,NaN,NaN,NINC,Ben,0
1,8425,555122,Expired close votes per date in range,NaN,DECLARE @startDate date = '##startDate##';\nD...,2017-05-18 2:42:04,NaN,NaN,NEC,Ben,1
2,985,190620,enumerate links for a post,NaN,"SELECT \n PostId as [Post Link],\n LinkTypeI...",2014-05-06 12:08:33,NaN,NaN,C,Ben,1
3,9186,172395,Find comments from terdon to the mirror,NaN,"SELECT \n id, PostId, Text \nFROM\n Comm...",2014-03-03 23:26:21,NaN,NaN,NINC,Ben,0
4,2788,559285,Users with their own Gold Tag Badges,NaN,"SELECT B.Name, B.UserId AS [User Link]\nFROM ...",2016-10-18 18:38:29,NaN,NaN,NIC,Ben,0
...,...,...,...,...,...,...,...,...,...,...,...
2007,9004,367001,"Separate a post in id, title, question and ans...","\nExample input: 3006643,2106715,1429318 \n(.c...","SELECT \n p.Id, p.Title, p.Body, k.Body\nFR...",2016-04-12 21:06:00,NaN,NaN,C,Moshe,0
2008,4673,467693,A list of the top 20 users with the highest av...,A list of the top 500 users with the highest a...,SELECT \n TOP 20\n Users.Id as [User Li...,2019-06-22 12:13:58,NaN,NaN,C,Vibhor,0
2009,1148,180327,How many posts have I edited?,NaN,SELECT Posts.LastEditorUserId AS [UserId]\n ...,2014-08-01 20:52:13,NaN,NaN,C,Vibhor,0
2010,6991,82274,What tag generates most reputation?,NaN,"SELECT \n TagName,\n CAST(SUM(Posts.Scor...",2013-05-10 13:17:07,NaN,NaN,C,Vibhor,0


In [62]:
df['title_nums']=df['Title'].str.extractall(r'(\d+)').groupby(level=0)[0].apply(list)

In [63]:
df['QueryBody']=df['QueryBody'].apply(lambda x:split_sql_text(x))

sp_help
sp_help
-- Enter Query Title
-- Enter Query Description

sp_who
-- Enter Query Title
-- Enter Query Description
sp_help Posts
-- Enter Query Title
-- Enter Query Description
sp_help Posts
-- Enter Query Title
-- Enter Query Description
sp_help Posts
-- Enter Query Title
-- Enter Query Description
sp_help posts
-- Enter Query Title
-- Enter Query Description

sp_columns Posts
-- Enter Query Title
-- Enter Query Description

sp_columns Posts
exec sp_columns Users
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Description
-- Enter Query Title
-- Enter Query Descriptio

print 'rajj'
EXEC sp_helpindex 'Posts'
--
exec sp_columns PostTypes;
sp_help users
exec sp_columns Posts;
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
--
sp_spaceused
-- Enter Query Title
-- Enter Query Description
sp_helpdb
-- Enter Query Title
-- Enter Query Description
sp_helpdb
sp_spaceused
exec sp_columns Badges;
exec sp_columns [users]
-- Let's try this again
-- Screw it.
sp_help Users
sp_tables
sp_databases
sp_databases
sp_who2
sp_who2
sp_help Tags
sp_tables
sp_databases
sp_help posts;
xp_msver
-- Top 50 Most Prolific Editors
-- Shows the top 50 post editors, where the user was the most recent editor
-- (meaning the results are conservative compared to the actual number of edits).

EXEC sp_databases
-- Top 50 Most Prolific Editors
-- Shows the top 50 post editors, where the user was the most recent editor
-- (meaning the results are conservative compared to the actual number of edits).

EXEC sp_helpdb
exec sp_who2
-- Enter Query Title grooping 
-- Enter Q

print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)

print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)

print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)

print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)

print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print replicate('x',135)+char(13)

print replicate('x',135)+char(13)
print replicate('x',135)+char(13)
print rep

print newid()
print @@CPU_BUSY
 print @@TIMETICKS
print @@CPU_BUSY
 print @@TIMETICKS
print convert(tinyint,rand()*5)+1
print choose(convert(tinyint,rand()*5)+1,'a','b','c','d','e')
print choose(convert(tinyint,rand()*5)+1,'a','b','c','d','e')
##Query##
##Query##
print('hi')
exec sp_help 'comments'
exec sp_help 'CloseAsOffTopicReasonTypes'
exec sp_help 'FlagTypes'
--- yeahh, this didn't work out
---
-- migrated to https://data.stackexchange.com/meta.stackoverflow/query/1012928/
--PRINT 2.*9*267509019

print 267509019/3
--PRINT 2.*9*267509019

print 267509019/13
--PRINT 2.*9*267509019

print 267509019/13
--4,8,15,16,23,42
print 20577616 * 13
---
-- migrated to https://data.stackexchange.com/meta.stackoverflow/query/1012928/
DBCC SHOW_STATISTICS ( 'dbo.Votes', _WA_Sys_00000003_3A81B327); 
DBCC SHOW_STATISTICS ( 'dbo.Votes', _WA_Sys_00000002_3A81B327); 
DBCC SHOW_STATISTICS ( 'dbo.Votes', _WA_Sys_00000006_3A81B327); 
DBCC SHOW_STATISTICS ( 'dbo.Votes', _WA_Sys_00000004_3A81B327); 
DBCC SH

In [64]:
df.QueryBody=df.QueryBody.dropna().apply(lambda x:x[1])

In [65]:
df.title_nums=df.title_nums.fillna('None')

In [66]:
df=df[~df['QueryBody'].isna()]

In [67]:
def query_contains_num(df):
    out=[]
    for row in df.itertuples():
        if row.title_nums == 'None':
            out.append(0)
        else:
            flag=True
#             print(row.title_nums)
            for i in row.title_nums:
                if i not in row.QueryBody:
                    out.append(1)
                    flag=False
                    break
            if flag:
                out.append(0)
    return out
                

df['title_num_flag']=query_contains_num(df)

In [68]:
df=df[df.title_num_flag==0]

In [69]:
df

,QuerySetId,Title,Description,QueryBody,CreationDate,Views,Votes,title_nums,title_num_flag
0,14342,What's my Pundit badge progress?,How many of my comments have scores of 5 and h...,DECLARE @UserId INT = ##UserId##;\n\nSELECT S...,2010-11-17 09:20:44,41810,30,None,0
9,139893,Questions by tags,NaN,SELECT * from posttags,2013-09-30 04:27:43,3510,0,None,0
12,14517,Keystrokes per rep point (lower limit),Ranks users with over 1000 reputation points b...,WITH over 5000 reputation points by how many ...,2010-11-20 00:38:02,2838,0,None,0
13,14542,Nick Craver comparison.,What questions does Nick Craver have a higher/...,DECLARE @UserId int = ##UserId##;\n\nwith fig...,2010-11-20 20:34:20,2522,1,None,0
14,14557,"Latest 13 ""possibly duplicate"" C++ questions",Delves into history records with type id 5. I ...,"SELECT top 13 Posts.Id as [Post Link], Posts....",2010-11-20 22:54:18,3139,0,[13],0
...,...,...,...,...,...,...,...,...,...
1574068,1207664,Questions with clarifying questions,NaN,"SELECT TOP 100 Question.Id AS [Post Link],\n ...",2020-03-08 20:58:25,37,0,None,0
1574077,1213051,teste,NaN,"SELECT Title,tags from Posts \ninner join Pos...",2020-12-10 20:55:24,59,0,None,0
1574092,1259299,svn,NaN,"SELECT top 100 p.id LongId, '>' CopyHelp, p.i...",2020-11-09 22:39:29,1133,1,None,0
1574093,1296007,Help Vampires - Users who amass reputation by ...,NaN,"SELECT top 10 U.Id as UserId, U.DisplayName a...",2020-11-09 23:00:35,78,0,None,0


In [70]:
with open('../stackexchange_schema/tables_so.json') as file:
    data=json.load(file)

In [71]:
columns=[]
for col in data[0]['column_names']:
    columns.append(col[1])

In [72]:
#title contains contain
def filter_title_query_contains(df, str1, str2):
    temp = df[df.Title.str.contains(str1, case=False)]
    df=df[~df.Title.str.contains(str1, case=False)]
    df=df.append(temp[temp.QueryBody.str.contains(str2, case=False)])
    return df

In [73]:
df = filter_title_query_contains(df, 'contains', 'LIKE')

In [74]:
if "*" in columns:
    columns.remove("*")
columns = list(set(columns))

In [75]:
for column in columns:
    print(column)
    df = filter_title_query_contains(df, column, column)

ReviewTaskId
DeactivationDate
DeletionUserId
CloseAsOffTopicReasonTypeId
IsAnonymous
UserId
RejectionReasonId
Description
OwnerDisplayName
PostHistoryTypeId
TargetUserId
AcceptedAnswerId
UserDisplayName
PostId
LastEditorDisplayName
PostNoticeTypeId
DeactivationModeratorId
CreationModeratorId
Body
Title
TargetRepChange
LastEditDate
ParentId
DownVotes
ContentLicense
Text
Predefined
UpVotes
ReviewTaskStateId
MarkdownConcensusDescription
Date
CommunityOwnedDate
InputTitle
PostNoticeDurationId
Id
ExpiryDate
ClassId
OwnerUserId
LastActivityDate
AboutMe
CreationDate
LastAccessDate
ReviewTaskTypeId
AccountId
CompletedByReviewTaskId
WebsiteUrl
Reputation
Location
MarkdownPostOwnerGuidance
RejectionDate
TagName
WikiPostId
DuplicateOfQuestionId
Tags
RevisionGUID
IsUniversal
ClosedDate
BountyAmount
TargetTagName
CommentCount
PostTypeId
TagId
LinkTypeId
ApprovalDate
ExcerptPostId
EmailHash
VoteTypeId
Score
LastAutoRename
BelongsOnBaseHostAddress
IsHidden
ApprovalModeratorId
CloseReasonTypeId
Commen

In [76]:
df.head()

,QuerySetId,Title,Description,QueryBody,CreationDate,Views,Votes,title_nums,title_num_flag
0,14342,What's my Pundit badge progress?,How many of my comments have scores of 5 and h...,DECLARE @UserId INT = ##UserId##;\n\nSELECT S...,2010-11-17 09:20:44,41810,30,None,0
12,14517,Keystrokes per rep point (lower limit),Ranks users with over 1000 reputation points b...,WITH over 5000 reputation points by how many ...,2010-11-20 00:38:02,2838,0,None,0
13,14542,Nick Craver comparison.,What questions does Nick Craver have a higher/...,DECLARE @UserId int = ##UserId##;\n\nwith fig...,2010-11-20 20:34:20,2522,1,None,0
14,14557,"Latest 13 ""possibly duplicate"" C++ questions",Delves into history records with type id 5. I ...,"SELECT top 13 Posts.Id as [Post Link], Posts....",2010-11-20 22:54:18,3139,0,[13],0
17,14647,Number of questions per day,NaN,"SELECT year(CreationDate), month(CreationDate...",2010-11-22 21:50:26,2918,1,None,0


In [77]:
df = df[df['Title'].str.count(" ") > 3]
df = df[df['QueryBody'].str.len() < 800]

In [78]:
df['top_num']=df['QueryBody'].apply(lambda x:re.findall("top (\w+)",x))

In [79]:
temp=df[df.top_num.apply(lambda x: len(x)) > 0]
df=df[df.top_num.apply(lambda x: len(x)) == 0]

In [80]:
temp.shape

(13047, 10)

In [81]:
df.shape

(126637, 10)

In [82]:
temp['top_num']=temp.top_num.apply(lambda x:x[0])

In [83]:
def query_contains_num(df):
    out=[]
    for row in df.itertuples():
        if row.top_num not in row.Title:
            out.append(1)#incorrect
        else:
            out.append(0)
    return out
                
temp['top_num_flag']=query_contains_num(temp)

In [84]:
del df["Views"]
del df["Votes"]
del df["title_nums"]
del df["top_num"]
del df["title_num_flag"]

In [85]:
df.head()

,QuerySetId,Title,Description,QueryBody,CreationDate
0,14342,What's my Pundit badge progress?,How many of my comments have scores of 5 and h...,DECLARE @UserId INT = ##UserId##;\n\nSELECT S...,2010-11-17 09:20:44
12,14517,Keystrokes per rep point (lower limit),Ranks users with over 1000 reputation points b...,WITH over 5000 reputation points by how many ...,2010-11-20 00:38:02
17,14647,Number of questions per day,NaN,"SELECT year(CreationDate), month(CreationDate...",2010-11-22 21:50:26
58,15490,Which of my answers are unsung?,Zero and non-zero accepted answers. Self-accep...,"SELECT \na.score, q.Title from\n Posts q\n ...",2010-12-01 22:03:01
65,15575,Users who post longest answers,Statistical summary of users who like to post ...,SELECT TOP 2000\n Rank = Row_Number() OVER...,2010-12-02 20:35:28


In [86]:
df=df.sort_values(['CreationDate']).drop_duplicates('Title',keep='last')

In [87]:
df

,QuerySetId,Title,Description,QueryBody,CreationDate
66942,1233,Top 500 Askers on the site,A list of the top 500 askers of questions\nord...,SELECT * FROM (\nSELECT \n TOP 500\n Ow...,2010-05-27 20:08:16
67055,1308,Average rep gain per post per tag,NaN,"SELECT t.tagName \n, COUNT(*) as NrOfQ...",2010-05-27 23:04:48
67099,1484,Badge CreationDate is always NULL (IS THIS A B...,NaN,"SELECT Name, (SELECT Count(*) FROM Badges B W...",2010-05-28 06:35:30
73305,1682,Check Votes.CreationDate is always a date,NaN,"SELECT \n Name, Count(*)as [Count], DatePa...",2010-05-29 03:25:23
73376,1703,Up vs Down votes by week day,NaN,"SELECT \n DATENAME(WEEKDAY, CreationDate) A...",2010-05-29 07:17:20
...,...,...,...,...,...
1273093,1051533,Get Tag Info for tags used over 50 times,NaN,"SELECT t.Id, t.TagName, ts.SourceTagName, t.C...",2020-12-10 15:53:21
1569475,1345529,select p1.id from (select * from Posts where ...,NaN,SELECT p1.id from (SELECT * from Posts where...,2020-12-10 17:32:25
118844,1346132,Percentage of SO Users on Meta,NaN,SELECT count(*) from users,2020-12-10 20:19:25
529921,1307450,Top 50 users from Rajkot,NaN,SELECT TOP 500\n ROW_NUMBER() OVER(ORDER B...,2020-12-10 20:34:32


In [88]:
df_with_annotated = pd.concat([df, filtered_annotated_df], axis=0)

In [89]:
df_with_annotated.drop_duplicates(["QuerySetId"], keep="last", inplace=True)

In [90]:
df_with_annotated

,QuerySetId,Title,Description,QueryBody,CreationDate,Unnamed: 0,Fixed Title,Fixed QueryBody,Type of Fix,Fixer,Faruk_Error
67099,1484,Badge CreationDate is always NULL (IS THIS A B...,NaN,"SELECT Name, (SELECT Count(*) FROM Badges B W...",2010-05-28 06:35:30,NaN,NaN,NaN,NaN,NaN,NaN
73305,1682,Check Votes.CreationDate is always a date,NaN,"SELECT \n Name, Count(*)as [Count], DatePa...",2010-05-29 03:25:23,NaN,NaN,NaN,NaN,NaN,NaN
73376,1703,Up vs Down votes by week day,NaN,"SELECT \n DATENAME(WEEKDAY, CreationDate) A...",2010-05-29 07:17:20,NaN,NaN,NaN,NaN,NaN,NaN
73458,1782,Up vs Down votes by hour of week of Question o...,NaN,SELECT \n CASE WHEN PostTypeId = 1 THEN 'Qu...,2010-05-29 17:58:51,NaN,NaN,NaN,NaN,NaN,NaN
73807,2305,Posts containing bit.ly shortened URLs,"Posts containing bit.ly shortened URLs, which ...","SELECT Id as [Post Link], Body\nfrom Posts \n...",2010-06-05 16:54:37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2007,367001,"Separate a post in id, title, question and ans...","\nExample input: 3006643,2106715,1429318 \n(.c...","SELECT \n p.Id, p.Title, p.Body, k.Body\nFR...",2016-04-12 21:06:00,9004.0,NaN,NaN,C,Moshe,0.0
2008,467693,A list of the top 20 users with the highest av...,A list of the top 500 users with the highest a...,SELECT \n TOP 20\n Users.Id as [User Li...,2019-06-22 12:13:58,4673.0,NaN,NaN,C,Vibhor,0.0
2009,180327,How many posts have I edited?,NaN,SELECT Posts.LastEditorUserId AS [UserId]\n ...,2014-08-01 20:52:13,1148.0,NaN,NaN,C,Vibhor,0.0
2010,82274,What tag generates most reputation?,NaN,"SELECT \n TagName,\n CAST(SUM(Posts.Scor...",2013-05-10 13:17:07,6991.0,NaN,NaN,C,Vibhor,0.0


In [91]:
df_with_annotated.shape

(12358, 11)

In [92]:
## test/val data for annotation
if "Unnamed: 0" in df_with_annotated.columns:
    df_with_annotated.drop(["Unnamed: 0"], axis=1, inplace=True)

val_test_df = df_with_annotated[~df_with_annotated["Fixer"].isna()]

In [94]:
print(f"Already annotated samples: {len(val_test_df)}")

Already annotated samples: 2012


In [95]:
VAL_TEST_SIZE = max(2000, len(val_test_df))

In [96]:
df_with_annotated = df_with_annotated.drop(val_test_df.index)

In [101]:
if VAL_TEST_SIZE - len(val_test_df) > 0:
    remaining_val_test_df = df_with_annotated.sample(n=VAL_TEST_SIZE - len(val_test_df), random_state=2021)
    df_filtered_test_val_annotated = pd.concat([val_test_df, remaining_val_test_df], axis=0)
    df_with_annotated = df_with_annotated.drop(remaining_val_test_df.index)
else:
    df_filtered_test_val_annotated = val_test_df.copy()

In [102]:
df_filtered_test_val_annotated

,QuerySetId,Title,Description,QueryBody,CreationDate,Fixed Title,Fixed QueryBody,Type of Fix,Fixer,Faruk_Error
0,1337675,Number of Help and Improvement Queries,NaN,"SELECT CreationDate, COUNT(*) FROM ReviewTask...",2020-11-28 16:13:32,NaN,NaN,NINC,Ben,0.0
1,555122,Expired close votes per date in range,NaN,DECLARE @startDate date = '##startDate##';\nD...,2017-05-18 2:42:04,NaN,NaN,NEC,Ben,1.0
2,190620,enumerate links for a post,NaN,"SELECT \n PostId as [Post Link],\n LinkTypeI...",2014-05-06 12:08:33,NaN,NaN,C,Ben,1.0
3,172395,Find comments from terdon to the mirror,NaN,"SELECT \n id, PostId, Text \nFROM\n Comm...",2014-03-03 23:26:21,NaN,NaN,NINC,Ben,0.0
4,559285,Users with their own Gold Tag Badges,NaN,"SELECT B.Name, B.UserId AS [User Link]\nFROM ...",2016-10-18 18:38:29,NaN,NaN,NIC,Ben,0.0
...,...,...,...,...,...,...,...,...,...,...
2007,367001,"Separate a post in id, title, question and ans...","\nExample input: 3006643,2106715,1429318 \n(.c...","SELECT \n p.Id, p.Title, p.Body, k.Body\nFR...",2016-04-12 21:06:00,NaN,NaN,C,Moshe,0.0
2008,467693,A list of the top 20 users with the highest av...,A list of the top 500 users with the highest a...,SELECT \n TOP 20\n Users.Id as [User Li...,2019-06-22 12:13:58,NaN,NaN,C,Vibhor,0.0
2009,180327,How many posts have I edited?,NaN,SELECT Posts.LastEditorUserId AS [UserId]\n ...,2014-08-01 20:52:13,NaN,NaN,C,Vibhor,0.0
2010,82274,What tag generates most reputation?,NaN,"SELECT \n TagName,\n CAST(SUM(Posts.Scor...",2013-05-10 13:17:07,NaN,NaN,C,Vibhor,0.0


In [103]:
df_filtered_test_val_annotated.sort_values(by=["QuerySetId"], inplace=True)
df_filtered_test_val_annotated.to_csv('../data/DataExp4_filtered_test_val_annotated.csv', index=False)

In [104]:
df_with_annotated.sort_values(by=["QuerySetId"], inplace=True)
df_with_annotated.to_csv('../data/DataExp4_filtered_not_annotated.csv', index=False)

/home/moshe/repos/sede/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
